In [1]:
from pymongo import MongoClient

# 连接到本机默认MongoDB服务
client = MongoClient()


In [2]:
# 连接或创建示例数据库
db = client['demo']


In [3]:
# 连接或创建示例集合
collection = db['demo_collection']


- 增

In [4]:
# 插入一条数据
collection.insert_one(
    {'name': '小明', 'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}}
)


In [5]:
# 插入多条数据
collection.insert_many(
    [
        {'name': name, 'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}}
        for name in ['小王', '小张', '小李']
    ]
)


- 查

In [6]:
# find()不直接返回结果，可以转list获取查询结果
list(collection.find({}))


[{'_id': ObjectId('64217bb51cee1325d080bed5'),
  'name': '小明',
  'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}},
 {'_id': ObjectId('64217bb51cee1325d080bed6'),
  'name': '小王',
  'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}},
 {'_id': ObjectId('64217bb51cee1325d080bed7'),
  'name': '小张',
  'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}},
 {'_id': ObjectId('64217bb51cee1325d080bed8'),
  'name': '小李',
  'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}}]

In [7]:
list(
    collection.find({
        'name': '小明'
    })
)

[{'_id': ObjectId('64217bb51cee1325d080bed5'),
  'name': '小明',
  'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}}]

In [8]:
# 嵌套字段判断
list(
    collection.find({
        'fields.a': 1
    })
)

[{'_id': ObjectId('64217bb51cee1325d080bed5'),
  'name': '小明',
  'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}},
 {'_id': ObjectId('64217bb51cee1325d080bed6'),
  'name': '小王',
  'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}},
 {'_id': ObjectId('64217bb51cee1325d080bed7'),
  'name': '小张',
  'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}},
 {'_id': ObjectId('64217bb51cee1325d080bed8'),
  'name': '小李',
  'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}}]

In [9]:
# 以小于等于为例
list(
    collection.find({
        'fields.a': {
            '$lte': 1
        }
    })
)

[{'_id': ObjectId('64217bb51cee1325d080bed5'),
  'name': '小明',
  'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}},
 {'_id': ObjectId('64217bb51cee1325d080bed6'),
  'name': '小王',
  'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}},
 {'_id': ObjectId('64217bb51cee1325d080bed7'),
  'name': '小张',
  'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}},
 {'_id': ObjectId('64217bb51cee1325d080bed8'),
  'name': '小李',
  'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}}]

In [10]:
# 查询目标数组长度不为0的记录
list(
    collection.find({
        'fields.c': {
            '$not': {
                '$size': 0
            }
        }
    })
)


[{'_id': ObjectId('64217bb51cee1325d080bed5'),
  'name': '小明',
  'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}},
 {'_id': ObjectId('64217bb51cee1325d080bed6'),
  'name': '小王',
  'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}},
 {'_id': ObjectId('64217bb51cee1325d080bed7'),
  'name': '小张',
  'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}},
 {'_id': ObjectId('64217bb51cee1325d080bed8'),
  'name': '小李',
  'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}}]

In [11]:
list(
    collection
    .find(
        {
            'name': {
                '$in': ['小明', '小张']
            }
        },
        # 去除自带的_id字段和fields.b字段
        {
            '_id': 0,
            'fields.b': 0
        }
    )
)

[{'name': '小明', 'fields': {'a': 1, 'c': [1, 2, 3]}},
 {'name': '小张', 'fields': {'a': 1, 'c': [1, 2, 3]}}]

- 改

In [12]:
# 更新指定字段值
(
    collection
    .update_many(
        # 更新条件
        {
            'name': {
                '$in': ['小明', '小张']
            }
        },
        # 更新内容
        {
            '$set': {
                'fields.a': 999
            }
        }
    )
)


In [13]:
list(
    collection.find(
        {
            'name': {
                '$in': ['小明', '小张']
            }
        }
    )
)


[{'_id': ObjectId('64217bb51cee1325d080bed5'),
  'name': '小明',
  'fields': {'a': 999, 'b': 'xxx', 'c': [1, 2, 3]}},
 {'_id': ObjectId('64217bb51cee1325d080bed7'),
  'name': '小张',
  'fields': {'a': 999, 'b': 'xxx', 'c': [1, 2, 3]}}]

In [14]:
# 为目标数组追加值
(
    collection
    .update_many(
        # 更新条件
        {
            'name': {
                '$in': ['小明', '小张']
            }
        },
        # 更新内容
        {
            '$push': {
                'fields.c': 999
            }
        }
    )
)

In [15]:
list(
    collection.find(
        {
            'name': {
                '$in': ['小明', '小张']
            }
        }
    )
)

[{'_id': ObjectId('64217bb51cee1325d080bed5'),
  'name': '小明',
  'fields': {'a': 999, 'b': 'xxx', 'c': [1, 2, 3, 999]}},
 {'_id': ObjectId('64217bb51cee1325d080bed7'),
  'name': '小张',
  'fields': {'a': 999, 'b': 'xxx', 'c': [1, 2, 3, 999]}}]

- 删

In [16]:
(
    collection
    .delete_many(
        {
            'name': '小张'
        }
    )
)

In [17]:
list(collection.find({}))

[{'_id': ObjectId('64217bb51cee1325d080bed5'),
  'name': '小明',
  'fields': {'a': 999, 'b': 'xxx', 'c': [1, 2, 3, 999]}},
 {'_id': ObjectId('64217bb51cee1325d080bed6'),
  'name': '小王',
  'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}},
 {'_id': ObjectId('64217bb51cee1325d080bed8'),
  'name': '小李',
  'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}}]

In [18]:
# 为name字段创建唯一值索引，切记一定要设置background=True从而更加稳定
collection.create_index('name', unique=True, background=True)


'name_1'

In [19]:
# 尝试插入name重复的数据
collection.insert_one(
    {'name': '小明', 'fields': {'a': 1, 'b': 'xxx', 'c': [1, 2, 3]}}
)


DuplicateKeyError: E11000 duplicate key error collection: demo.demo_collection index: name_1 dup key: { name: "小明" }, full error: {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: demo.demo_collection index: name_1 dup key: { name: "小明" }', 'keyPattern': {'name': 1}, 'keyValue': {'name': '小明'}}

In [20]:
# 基于JSONSchema创建文档格式校验规则
# 删除示例collection
db.drop_collection('demo_collection')
db.create_collection(
    'demo_collection',
    validator={
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["name", "fields"],
            "properties": {
                "name": {
                    "bsonType": "string"
                },
                "fields": {
                    "bsonType": "object",
                    "required": ["a", "b"]
                }
            }
        }
    }
)
collection = db['demo_collection']


In [21]:
# 尝试插入不合规数据
try:
    collection.insert_one(
        {
            'name': '小明'
        }
    )
except Exception as e:
    print('插入失败')
    print(e)

插入失败
Document failed validation, full error: {'index': 0, 'code': 121, 'errmsg': 'Document failed validation', 'errInfo': {'failingDocumentId': ObjectId('64217bc11cee1325d080beda'), 'details': {'operatorName': '$jsonSchema', 'schemaRulesNotSatisfied': [{'operatorName': 'required', 'specifiedAs': {'required': ['name', 'fields']}, 'missingProperties': ['fields']}]}}}


In [22]:
# 插入合规数据
collection.insert_one(
    {
        'name': '小明',
        'fields': {
            'a': 1,
            'b': 1
        }
    }
)
